INSTALLATIONS:

1.python3                                                                                                  
2.pip                                                                                                      
3.pkg-config                                                                                               
4.OpenCV    
5.scikit-learn                                                                                      

1.Clone darknet repository (https://github.com/AlexeyAB/darknet) in darknet directory


2.Not using GPU/CUDA for running

In [9]:
import os

os.chdir("/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING/darknet/darknet")

# Modify the Makefile using Python
import re

# Function to update the Makefile
def update_makefile(path):
    with open(path, 'r') as file:
        content = file.read()
    
    content = re.sub(r'^OPENCV=.*', 'OPENCV=0', content, flags=re.MULTILINE)
    content = re.sub(r'^GPU=.*', 'GPU=0', content, flags=re.MULTILINE)
    content = re.sub(r'^CUDNN=.*', 'CUDNN=0', content, flags=re.MULTILINE)
    content = re.sub(r'^CUDNN_HALF=.*', 'CUDNN_HALF=0', content, flags=re.MULTILINE)
    content = re.sub(r'^LIBSO=.*', 'LIBSO=0', content, flags=re.MULTILINE)
    
    with open(path, 'w') as file:
        file.write(content)

# Call the function to update the Makefile
update_makefile("/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING/darknet/darknet/Makefile")

3.Setup yolov4-tiny directory (same as training)

4.Build darknet

In [ ]:
import subprocess
import os

# Change to the directory containing the Makefile
os.chdir("/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING/darknet/darknet")

try:
    # Run the make clean command first
    clean_result = subprocess.run('make clean', shell=True, capture_output=True, text=True, check=True)
    print("Make Clean STDOUT:\n", clean_result.stdout)

    # Run the make command
    result = subprocess.run('make', shell=True, capture_output=True, text=True, check=True)
    print("Make STDOUT:\n", result.stdout)

except subprocess.CalledProcessError as e:
    # Print standard output and error if any of the commands fail
    print("STDOUT:\n", e.stdout)
    print("STDERR:\n", e.stderr)
    print("Error code:", e.returncode)

5.Verify Darknet build and copy files to the data directory

In [11]:
import subprocess
import os

os.chdir("/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING/darknet/darknet")

# Verify the build by checking the darknet executable
subprocess.run(['ls', '-l', 'darknet'])

# Change permissions to make darknet executable
subprocess.run(['chmod', '+x', 'darknet'])

# Run a simple test to ensure Darknet is working
subprocess.run(['./darknet'])


-rwxrwxrwx 1 rayaan rayaan 1110944 Aug 17 17:49 darknet


usage: ./darknet <function>


CompletedProcess(args=['./darknet'], returncode=0)

In [12]:
import subprocess
import os
import shutil
import zipfile

# Define the base paths for the source and destination directories
base_dir = '/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING'
darknet_dir = os.path.join(base_dir, 'darknet/darknet')
yolov4_tiny_dir = os.path.join(base_dir, 'yolov4-tiny')
obj_zip_path = os.path.join(yolov4_tiny_dir, 'obj.zip')
data_dir = os.path.join(darknet_dir, 'data')

# Change to the 'data' directory and remove all files except the 'labels' folder
os.chdir(data_dir)
subprocess.run('find . -maxdepth 1 -type f ! -name "labels" -exec rm -rf {} \;', shell=True)

# Remove the 'cfg' directory and recreate it
os.chdir(darknet_dir)
subprocess.run('rm -rf cfg', shell=True)
os.mkdir('cfg')

# Remove the 'obj' directory in the 'data' directory if it exists
obj_dir = os.path.join(data_dir, 'obj')
if os.path.exists(obj_dir):
    shutil.rmtree(obj_dir)

# Unzip obj.zip into the 'data' directory
with zipfile.ZipFile(obj_zip_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

# Copy the custom cfg file to the 'cfg' folder
subprocess.run(f'cp {os.path.join(yolov4_tiny_dir, "yolov4-tiny-custom.cfg")} ./cfg', shell=True)

# Copy the obj.names and obj.data files to the 'data' folder
subprocess.run(f'cp {os.path.join(yolov4_tiny_dir, "obj.names")} ./data', shell=True)
subprocess.run(f'cp {os.path.join(yolov4_tiny_dir, "obj.data")} ./data', shell=True)

print("Setup complete.")

Setup complete.


6.Generate train.txt and test.txt

In [13]:
import os
import shutil
import zipfile
from sklearn.model_selection import train_test_split

# Define the base paths for the source and destination directories
base_dir = '/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING'
darknet_dir = os.path.join(base_dir, 'darknet/darknet')
yolov4_tiny_dir = os.path.join(base_dir, 'yolov4-tiny')
obj_zip_path = os.path.join(yolov4_tiny_dir, 'obj.zip')
data_dir = os.path.join(darknet_dir, 'data')
obj_dir = os.path.join(data_dir, 'obj')

# Remove the 'obj' directory in the 'data' directory if it exists
if os.path.exists(obj_dir):
    shutil.rmtree(obj_dir)

# Unzip obj.zip into the 'data' directory
with zipfile.ZipFile(obj_zip_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

# Create paths for cfg files
os.makedirs(os.path.join(darknet_dir, 'cfg'), exist_ok=True)

# Copy the custom cfg file to the 'cfg' folder
shutil.copy(os.path.join(yolov4_tiny_dir, "yolov4-tiny-custom.cfg"), os.path.join(darknet_dir, 'cfg'))

# Copy the obj.names and obj.data files to the 'data' folder
shutil.copy(os.path.join(yolov4_tiny_dir, "obj.names"), data_dir)
shutil.copy(os.path.join(yolov4_tiny_dir, "obj.data"), data_dir)

# Create train.txt and test.txt
image_files = [f for f in os.listdir(obj_dir) if os.path.isfile(os.path.join(obj_dir, f))]
image_paths = [os.path.join('/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING/darknet/darknet/data/obj', f) for f in image_files]

# Split the data into training and testing sets
train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)

# Save the paths to train.txt and test.txt in the 'data' directory
with open(os.path.join(data_dir, 'train.txt'), 'w') as f:
    f.write('\n'.join(train_paths) + '\n')

with open(os.path.join(data_dir, 'test.txt'), 'w') as f:
    f.write('\n'.join(test_paths) + '\n')

print("Setup complete.")


Setup complete.


7.Verify cfg file is set to running mode -               
  batch=1                                              
  subdivisions=1

In [15]:
import subprocess
import os

# Change directory to 'cfg'
os.chdir("/mnt/c/Users/satar/OneDrive/Desktop/KEY_FRAME_SELECTION/RUNNING/darknet/darknet/cfg")

# Update the configuration file using sed commands
subprocess.run(['sed', '-i', 's/batch=64/batch=1/', 'yolov4-tiny-custom.cfg'])
subprocess.run(['sed', '-i', 's/subdivisions=16/subdivisions=1/', 'yolov4-tiny-custom.cfg'])

CompletedProcess(args=['sed', '-i', 's/subdivisions=16/subdivisions=1/', 'yolov4-tiny-custom.cfg'], returncode=0)